In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import re
import nltk
import joblib

In [4]:
# Fonction de prétraitement du texte
def preprocess_text(text):
    text = text.lower()  # Convertir en minuscule
    text = re.sub(r'http\S+', '', text)  # Supprimer les liens
    text = re.sub(r'@\w+', '', text)  # Supprimer les mentions
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères non-alphabétiques
    return text

# Charger le dataset CSV
df = pd.read_csv('dataset/phrases_emojis.csv')  # Remplacez par le chemin de votre fichier CSV

# Prétraiter les tweets
df['tweet_processed'] = df['Phrase'].apply(preprocess_text)

In [9]:
# Tokenisation des tweets
tokenized_tweets = [word_tokenize(tweet) for tweet in df['tweet_processed']]

# Entraîner un modèle Word2Vec
model = Word2Vec(sentences=tokenized_tweets, window=5, min_count=1, sg=0)

# Sauvegarder le modèle
model.save("models/word2vec_model_twitter.model")
print("Modèle sauvegardé.")

Modèle sauvegardé.


In [12]:
# Fonction pour obtenir l'embedding moyen d'un tweet
def get_tweet_embedding(tweet, model):
    tokens = word_tokenize(preprocess_text(tweet))  # Tokenisation
    embeddings = []
    
    for word in tokens:
        if word in model.wv:
            embeddings.append(model.wv[word])
    
    if embeddings:
        return np.mean(embeddings, axis=0)  # Renvoie l'embedding moyen
    else:
        return np.zeros(model.vector_size)  # Si aucun mot trouvé dans le modèle, renvoie un vecteur nul

# Extraire les embeddings des tweets
X = np.array([get_tweet_embedding(tweet, model) for tweet in df['Phrase']])

# Encoder les emojis en labels numériques
encoder = LabelEncoder()
df['emoji_label'] = encoder.fit_transform(df['Emojis'])

# Définir les labels (y) et les caractéristiques (X)
y = df['emoji_label'].values

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, np.arange(len(df)), test_size=0.2, random_state=42)

# Entraîner un modèle de classification (RandomForest ici, mais vous pouvez essayer d'autres modèles)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)


RandomForestClassifier()

In [14]:
# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluation de la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy * 100:.2f}%")

# Afficher des emojis prédits pour quelques exemples
predicted_emojis = encoder.inverse_transform(y_pred)

# Récupérer les tweets et emojis réels correspondants aux indices de l'ensemble de test
tweets_predicted = df.iloc[test_indices]['Phrase']
emojis_real = df.iloc[test_indices]['Emojis']

# Afficher les résultats : tweet, emoji réel et emoji prédit
for tweet, emoji_real, emoji_pred in zip(tweets_predicted.head(30), emojis_real.head(30), predicted_emojis[:30]):
    print(f"Tweet: {tweet}\nEmoji réel: {emoji_real}\nEmoji prédit: {emoji_pred}\n")


Précision du modèle : 54.92%
Tweet: Après avoir progressivement reculer l'heure de mon réveil⏰ pour m'y habituer pas à pas, je me lève tous les jours à 5h30  .
Emoji réel: ✅🕠
Emoji prédit: ✅🕠

Tweet: Ah et j'ai aussi peur de ne pas comprendre un mot important dans la question  ça m'a arrivé au bac blanc pour l'essai philosophique rip
Emoji réel: 👍
Emoji prédit: 😮💨

Tweet: Est ce qu’il est possible de faire quelque chose de plus? Est-ce possible d’être de faire de la désensibilisation? Des conseils?
Emoji réel: 😭
Emoji prédit: 😷

Tweet: Bonjour
Emoji réel: 👋🏻
Emoji prédit: 👋

Tweet: Bonne soirée redditeurs / redditrices.
Emoji réel: ❤️
Emoji prédit: ❤️

Tweet: Merci d’avance pour votre aide !  (merci de me mentionner lors de vos réponse !)
Emoji réel: 🙏🔥
Emoji prédit: 🙏🔥

Tweet: question concernant ma merguez bleue
Emoji réel: 😂
Emoji prédit: 😅

Tweet: (Potentiellement un dernier édit à venir si je ne croise personne à la gare )
Emoji réel: 😭
Emoji prédit: 🤔

Tweet: Bref, une solution p